In [3]:
from __future__ import print_function
import keras
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
import numpy as np
import os


In [4]:
from keras import utils
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

In [5]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
# Normalizing the data.
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

170508288/170498071 [==============================] - 2s 0us/step


In [6]:
def identity_block(X, f, filters, stage, block):
    F1, F2, F3 = filters 
    X_shortcut = X

    # First component of main path
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    # Second component of main path 
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    # Third component of main path 
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid')(X)
    X = BatchNormalization(axis=3)(X)

    # Final step: Adding shortcut value to main path, and passing it through a RELU activation 
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

In [7]:
def convolutional_block(X, f, filters, stage, block, s=2):
    F1, F2, F3 = filters
    X_shortcut = X

    # First component of main path 
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(s, s), padding='valid')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid')(X)
    X = BatchNormalization(axis=3)(X)

    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid')(X_shortcut)
    X_shortcut = BatchNormalization(axis=3)(X_shortcut)

    # Adding shortcut value to main path, and passing it through a RELU activation 
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    return X

In [8]:
def ResNet50(input_shape=(32, 32, 3), classes=10):

    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)

    # Stage 1
    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    # Stage 3
    X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4
    X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5
    X = X = convolutional_block(X, f=3, filters=[512, 512, 2048], stage=5, block='a', s=2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL
    X = AveragePooling2D(pool_size=(2, 2), padding='same')(X)

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', kernel_initializer='he_normal')(X)

    # Creating model
    model = Model(inputs=X_input, outputs=X, name='ResNet50')

    return model

In [9]:
model = ResNet50(input_shape=(32, 32, 3), classes=10)

In [10]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
model.fit(x_train, y_train, epochs = 50, batch_size = 32)

Epoch 1/50
1563/1563 [==============================] - 62s 40ms/step - loss: 0.3856 - accuracy: 0.8670
Epoch 2/50
1563/1563 [==============================] - 63s 40ms/step - loss: 0.3156 - accuracy: 0.8897
Epoch 3/50
1563/1563 [==============================] - 63s 40ms/step - loss: 0.2774 - accuracy: 0.9022
Epoch 4/50
1563/1563 [==============================] - 63s 41ms/step - loss: 0.2604 - accuracy: 0.9082
Epoch 5/50
1563/1563 [==============================] - 63s 41ms/step - loss: 0.2002 - accuracy: 0.9287
Epoch 6/50
1563/1563 [==============================] - 64s 41ms/step - loss: 0.1853 - accuracy: 0.9360
Epoch 7/50
1563/1563 [==============================] - 64s 41ms/step - loss: 0.1564 - accuracy: 0.9448
Epoch 8/50
1563/1563 [==============================] - 64s 41ms/step - loss: 0.1437 - accuracy: 0.9511
Epoch 9/50
1563/1563 [==============================] - 64s 41ms/step - loss: 0.1387 - accuracy: 0.9518
Epoch 10/50
1563/1563 [==============================] - 64s 41m

In [16]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 4s 13ms/step - loss: 2.2366 - accuracy: 0.7429


[2.2366349697113037, 0.742900013923645]